In [23]:
# disable compiler warnings
import os

# imports 
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_text as tf_text
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras.layers import Dense
from typing import List
import datetime
from tqdm.notebook import tqdm


from tensorflow.python.client import device_lib
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'  # FATAL
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

ModuleNotFoundError: No module named 'tensorflow_text'

In [19]:
file_path = "bible.txt"

with open(file_path, "r") as f:
    text = f.read().lower()

splitter = RegexSplitter()

splitted = splitter(text)

tokenizer = tf.keras.preprocessing.text.Tokenizer(
                                                 num_words=None,
                                                 filters="\n!'()*,-.0123456789:;?",
                                                 lower=True,
                                                 split=' ',
                                                 char_level=False,
                                                 oov_token=None,
                                                 analyzer=None,
                                                 )

tokens = tokenizer.fit_on_texts([text])
print(tokens)

NameError: name 'RegexSplitter' is not defined

In [9]:

vocabulary = {x: i for i, x in enumerate(np.unique(list(text)))}
char_tokens = [vocabulary[char] for char in text]
print(vocabulary)

{'\n': 0, ' ': 1, '!': 2, "'": 3, '(': 4, ')': 5, '*': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '?': 22, 'a': 23, 'b': 24, 'c': 25, 'd': 26, 'e': 27, 'f': 28, 'g': 29, 'h': 30, 'i': 31, 'j': 32, 'k': 33, 'l': 34, 'm': 35, 'n': 36, 'o': 37, 'p': 38, 'q': 39, 'r': 40, 's': 41, 't': 42, 'u': 43, 'v': 44, 'w': 45, 'x': 46, 'y': 47, 'z': 48}


In [8]:
sequence_length = 4
dataset = tf.data.Dataset.from_tensor_slices(char_tokens)
dataset = dataset.window(sequence_length,drop_remainder=True)
batch_windows = lambda x: x.batch(sequence_length).get_single_element()
dataset = dataset.map(batch_windows)
dataset = dataset.shuffle(10000).batch(64).prefetch(tf.data.AUTOTUNE)
for seq in dataset.take(1):
    tf.print(seq)

[[26 1 24 27]
 [23 23 36 9]
 [23 43 29 30]
 ...
 [1 45 23 41]
 [1 25 23 34]
 [43 36 26 1]]
